# Task 4.8

In [5]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [6]:
# Import ords_prods_merged_new dataframe
path = r'C:\Users\daxma\OneDrive\Desktop\Data Analytics\Data Immersion\Achievement 4\01-2025 Instacart Basket Analysis'
file_path = os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merged_new.pkl')
ords_prods_merge = pd.read_pickle(file_path)

In [7]:
# Check output
ords_prods_merge.head(10)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order_flag,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,1,2,8,NaN,1,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Average orders,10,New customer
1,2539329,1,1,2,8,NaN,1,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both,Mid-range product,Regularly busy,Average orders,10,New customer
2,2539329,1,1,2,8,NaN,1,12427,3,0,Original Beef Jerky,23,19,4.4,both,Low-range product,Regularly busy,Average orders,10,New customer
3,2539329,1,1,2,8,NaN,1,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,both,Low-range product,Regularly busy,Average orders,10,New customer
4,2539329,1,1,2,8,NaN,1,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both,Low-range product,Regularly busy,Average orders,10,New customer
5,2398795,1,2,3,7,15.0,0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Slowest days,Average orders,10,New customer
6,2398795,1,2,3,7,15.0,0,10258,2,0,Pistachios,117,19,3.0,both,Low-range product,Slowest days,Average orders,10,New customer
7,2398795,1,2,3,7,15.0,0,12427,3,1,Original Beef Jerky,23,19,4.4,both,Low-range product,Slowest days,Average orders,10,New customer
8,2398795,1,2,3,7,15.0,0,13176,4,0,Bag of Organic Bananas,24,4,10.3,both,Mid-range product,Slowest days,Average orders,10,New customer
9,2398795,1,2,3,7,15.0,0,26088,5,1,Aged White Cheddar Popcorn,23,19,4.7,both,Low-range product,Slowest days,Average orders,10,New customer


In [8]:
# Step 2 Aggregating mean of order_number grouped by deparment_id
ords_prods_merge.groupby('department_id').agg({'order_number' : ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


### Step 3:
### There are slight variations in mean values when comparing the subset to the full dataframe. However, the top 3 values (deparment_id: 21, 18, 10) remain the same. The bottom 3 values still contain deparment_id: 1 and 5, but now include department id: 8 instead of department_id: 6.

### Step 4:
### I already created a loyalty flag for customers in this dataframe in the notebook from the exercise. I will include that notebook in my submission for you to view my work.

In [11]:
# Step 5 Check difference in spending by different loyalty tiers
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

### *Note: Fix max pricing error*

### Loyal customers seem to be spending, on average, less than new and regular customers. New customers, on average, spend the most. However, the max price seems to be incorrect so this conclusion may not be reliable until it is dealt with.

In [14]:
# Step 6 Calculate the average spending per user per product
ords_prods_merge['avg_product_price'] = ords_prods_merge.groupby('user_id')['prices'].transform(np.mean)

C:\Users\daxma\AppData\Local\Temp\ipykernel_22444\732068589.py:2: FutureWarning: The provided callable <function mean at 0x0000025FCCC29120> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['avg_product_price'] = ords_prods_merge.groupby('user_id')['prices'].transform(np.mean)


In [15]:
# Assign spending flag using loc
ords_prods_merge.loc[ords_prods_merge['avg_product_price'] < 10, 'product_spending_flag'] = 'Low product spender'
ords_prods_merge.loc[ords_prods_merge['avg_product_price'] >= 10, 'product_spending_flag'] = 'High product spender'

In [16]:
# Check the distribution
ords_prods_merge['product_spending_flag'].value_counts()

product_spending_flag
Low product spender     31770614
High product spender      634245
Name: count, dtype: int64

In [17]:
# Step 6 Continued Calculate total spending per order per user
order_totals = ords_prods_merge.groupby(['user_id', 'order_number'])['prices'].sum().reset_index()

In [18]:
# Calculate the average spending per order per user
user_avg_spending = order_totals.groupby('user_id')['prices'].mean().reset_index()
user_avg_spending.rename(columns={'prices': 'avg_order_spending'}, inplace=True)

In [19]:
# Merge the average spending back into the main dataframe
ords_prods_merge = ords_prods_merge.merge(user_avg_spending, on='user_id', how='left')

In [20]:
# Describe avg_user_spending across customers
ords_prods_merge.agg({'avg_order_spending': ['mean', 'median', 'min', 'max']})

,avg_order_spending
mean,159.133140
median,94.675000
min,1.000000
max,100177.830769


In [21]:
# Assign spending flags
ords_prods_merge.loc[ords_prods_merge['avg_order_spending'] < 160, 'order_spending_flag'] = 'Low order spender'
ords_prods_merge.loc[(ords_prods_merge['avg_order_spending'] >= 160) & (ords_prods_merge['avg_order_spending'] <5000), 'order_spending_flag'] = 'High order spender'
ords_prods_merge.loc[ords_prods_merge['avg_order_spending'] >= 5000, 'order_spending_flag'] = 'Value Error'

In [22]:
# Check the distribution
ords_prods_merge['order_spending_flag'].value_counts()

order_spending_flag
Low order spender     27825771
High order spender     4498471
Value Error              80617
Name: count, dtype: int64

### Note: I thought it would be valuable to also separate users on their average total order spending rather than just on average products purchased price

In [24]:
# Step 7 Compute the median days since prior order for each user
user_median_days = ords_prods_merge.groupby('user_id')['days_since_prior_order'].median()

In [25]:
# Merge the median values back into dataframe
ords_prods_merge = ords_prods_merge.merge(user_median_days.rename('median_days_since_prior_order'), on='user_id', how='left')

In [26]:
# Assign flags based on median days since prior order
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] > 20, 'order_freq_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_days_since_prior_order'] <= 20) & (ords_prods_merge['median_days_since_prior_order'] > 10), 'order_freq_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] <= 10, 'order_freq_flag'] = 'Frequent customer'

In [27]:
# Check the distribution
ords_prods_merge['order_freq_flag'].value_counts()

order_freq_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
Name: count, dtype: int64

In [28]:
# Export as pkl file
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merge_new_cols.pkl'))